In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Set environment variables
import os

os.environ["SCAL_TYPE"] = "real"
os.environ["PRECISION"] = "single"
os.environ["MY_NUMBA_TARGET"] = "numba"

# Add cle_fun to PYTHON_PATH
import sys
sys.path.append("../../clonscal")

In [2]:
from simulation.cl_simulation import ComplexLangevinSimulation
from simulation.config import Config

config = Config(dt = 0.01, dims = [100, 100])
sim = ComplexLangevinSimulation(config)

Using single precision
Using R^1
Using Numba


In [4]:
import inspect

def get_kernel_args(kernel_func, sim: ComplexLangevinSimulation):
    kernel_params = inspect.signature(kernel_func).parameters.keys()
    assert 'idx' in kernel_params, "Pass a kernel function with 'idx' argument"
    
    kernel_vals = {
    (key if key != "idx" else "n_cells"): sim.__dict__[key if key != "idx" else "n_cells"]
    for key in kernel_params
    }
    return kernel_vals

In [12]:
import inspect
from src.utils import mexican_hat_kernel_real, update_noise_kernel

from src.numba_target import my_parallel_loop
kernel_vals = get_kernel_args(update_noise_kernel, sim).values()
my_parallel_loop(update_noise_kernel, *kernel_vals)